# HW 3 Classification
28 Feb 2025



In [4]:
import ee
import geemap
import matplotlib.pyplot as plt
import numpy as np

In [5]:
ee.Authenticate()
ee.Initialize(project='ee-wongluyii')

### Landsat 8 Image Selection

In [8]:
# Modified Version
# Pulling Phildelphia boundary (Region of Interest)
philly = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq("NAME", "Philadelphia"))

# Load Landsat 8 Collection (TOA Reflectance)
landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
           .filterBounds(philly)
           .filterDate("2023-01-01", "2023-12-31")
           .filterMetadata("CLOUD_COVER", "less_than", 10)
           .median()
           .clip(philly))


# Visualize ROI and Landsat image
Map = geemap.Map()
Map.centerObject(philly, 10)

stats = landsat.reduceRegion(
    reducer=ee.Reducer.minMax(),
    geometry=philly,
    scale=30
)
min_value = stats.get("B4_min")
max_value = stats.get("B4_max")
Map.addLayer(landsat, {"bands": ["B4", "B3", "B2"], "min": 5, "max": max_value}, "True Color")
Map

Map(center=[40.00760439369441, -75.13403899470623], controls=(WidgetControl(options=['position', 'transparent_…

In [4]:
# Collect and Label Training Data
Map.draw_features

[]

In [ ]:
if Map.user_rois is not None:
    training_samples = Map.user_rois
    print(training_samples.getInfo())

In [5]:
Map.user_rois.getInfo()

{'type': 'FeatureCollection', 'columns': {}, 'features': []}

In [ ]:
geemap.ee_to_shp(Map.user_rois, "water_samples.shp")

In [3]:
# Export Image
# Pulling Phildelphia boundary (Region of Interest)
philly = ee.FeatureCollection("TIGER/2018/Counties").filter(ee.Filter.eq("NAME", "Philadelphia"))

# Load Landsat 8 Collection
landsat = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
           .filterBounds(philly)
           .filterDate("2023-01-01", "2023-12-31")
           .filterMetadata("CLOUD_COVER", "less_than", 10)
           .median()
           .clip(philly)
           .select("B[1-7]"))

# Define a boundary
small_area = philly.geometry().buffer(10)

# Export Landsat Image 
geemap.ee_export_image(landsat,  scale=30, filename="landsat_philly_2023_v2.tif", region=small_area)

Generating URL ...
An error occurred while downloading.
Total request size (76509846 bytes) must be less than or equal to 50331648 bytes.


In [ ]:
# Create an interactive Map
Map = geemap.Map(center=(39.9526, -75.1652), zoom=12)
Map.add_basemap('HYBRID')

<IPython.core.display.Javascript object>

vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'gamma': 1.0, 'min': 0.0, 'max': 0.3}


<IPython.core.display.Javascript object>

vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'gamma': 1.0, 'min': 0.0, 'max': 0.3}


No such comm: 076b46b8dde848779a95c2c37ddf4771
No such comm: 076b46b8dde848779a95c2c37ddf4771
No such comm: 076b46b8dde848779a95c2c37ddf4771
No such comm: 076b46b8dde848779a95c2c37ddf4771
No such comm: 076b46b8dde848779a95c2c37ddf4771


In [ ]:
# Load City Boundary
phil_boundary = object #load in boundary

dataset = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate("2023-01-01", "2023-12-31").filterBounds(phil_boundary).select("SR_B[1-7]")

# Set visualization parameters
vis_params = {"min": 0, "max": 3000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

In [43]:
# Landsat 8 Image Selection
# Selecting point geometry for Philadelphia (City Hall as a reference)
point = ee.Geometry.Point([-75.1652, 39.9526])

# Landsat 8 Level 2, Collection 2, Tier 1
# Date range Jan to Dec 2023, sorting by lowest cloud cover
# Select spectral bands 1-7
image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2023-01-01", "2023-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)


# Set visualization parameters
vis_params = {"min": 0, "max": 3000, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

# Add the Earth Engine Layers
Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")

# Visualize Interactive Map
Map

Map(bottom=397449.0, center=[39.9526, -75.1652], controls=(WidgetControl(options=['position', 'transparent_bg'…

In [70]:
# Landsat 8 Level 2, Collection 2, Tier 1
dataset = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
.filterDate('2021-05-01', '2021-06-01')

# Applies scaling factors.
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
    .addBands(thermalBands, None, True)

dataset = dataset.map(applyScaleFactors)

vis_params = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'gamma': 1.0, 'min': 0.0, 'max': 0.3}

Map.setCenter(-75.1652, 39.9526, 10) 

Map.addLayer(dataset, vis_params, 'Landsat8')

In [71]:
Map

Map(bottom=50087.0, center=[39.65011210186374, -74.70153808593751], controls=(WidgetControl(options=['position…

In [60]:
# NDVI Landsat 8 Collection 2 Tier 1 Level 2 8-day composite
NDVI = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI')
Map.addLayer(NDVI, {}, 'LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDVI')

# NDWI 
NDWI = ee.ImageCollection('LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI')
Map.addLayer(NDWI, {}, 'LANDSAT/COMPOSITES/C02/T1_L2_8DAY_NDWI')

In [62]:
# Downloading Google Earth Engine Catalog 
from geemap.datasets import DATA

### Check Image Properties

In [18]:
print("The Image Date is:", ee.Date(image.get("system:time_start")).format("YYYY-MM-dd").getInfo())
print("The cloud cover is:", image.get("CLOUD_COVER").getInfo())

The Image Date is: 2023-04-10
The cloud cover is: 0.08


### Preparing Training Dataset

In [19]:
# Creating a buffer 
# region = Map.user_roi
# region = ee.Geometry.Rectangle([-122.6003, 37.4831, -121.8036, 37.8288])
region = ee.Geometry.Point([-75.1652, 39.9526]).buffer(10000)


In [29]:
# NLCD Classification
nlcd = ee.Image("USGS/NLCD/NLCD2016").select("landcover").clip(image.geometry())
Map.addLayer(nlcd, {}, "NLCD")
Map

# Make the training dataset.
points = nlcd.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(points, {}, "training", False)

In [31]:
# Training the Classifier
# Use these bands for prediction.
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]


# This property of the table stores the land cover labels.
label = "landcover"

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'SR_B1': 8130, 'SR_B2': 8129, 'SR_B3': 8496, 'SR_B4': 8544, 'SR_B5': 10505, 'SR_B6': 12037, 'SR_B7': 10508, 'landcover': 42}}


In [32]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, "classified")
Map

Map(center=[39.9526, -75.1652], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…